In [27]:
import pandas as pd 
from tensorflow import keras
from keras.utils import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical, pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [28]:
import pandas as pd

# Load the dataset from Hugging Face
dataset =pd.read_parquet("hf://datasets/thesven/Reflective-MAGLLAMA-v0.1/data/train-00000-of-00001.parquet")


In [29]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10227 entries, 0 to 10226
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   instruction          10227 non-null  object
 1   model_name           10227 non-null  object
 2   generation           10227 non-null  object
 3   distilabel_metadata  10227 non-null  object
 4   text                 10227 non-null  object
 5   embedding            10227 non-null  object
 6   nn_indices           10227 non-null  object
 7   nn_scores            10227 non-null  object
 8   reflections          10227 non-null  object
dtypes: object(9)
memory usage: 719.2+ KB


In [30]:
import pandas as pd

# Assuming 'df' is your DataFrame
# Remove the first 6227 rows
df = dataset.iloc[10210:]

# Verify the shape of the DataFrame
print(df.shape)


(17, 9)


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 10210 to 10226
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   instruction          17 non-null     object
 1   model_name           17 non-null     object
 2   generation           17 non-null     object
 3   distilabel_metadata  17 non-null     object
 4   text                 17 non-null     object
 5   embedding            17 non-null     object
 6   nn_indices           17 non-null     object
 7   nn_scores            17 non-null     object
 8   reflections          17 non-null     object
dtypes: object(9)
memory usage: 1.3+ KB


In [32]:
df.describe()

,instruction,model_name,generation,distilabel_metadata,text,embedding,nn_indices,nn_scores,reflections
count,17,17,17,17,17,17,17,17,17
unique,17,1,17,17,17,17,17,17,17
top,Create a thinking map of the benefits and draw...,sentence-transformers/all-MiniLM-L6-v2,<thinking>\nI need to think about the context ...,{'raw_output_text_generation_0': '<thinking> I...,[{'content': 'Create a thinking map of the ben...,"[-0.04562043026089668, 0.020021433010697365, 0...",[5381],[0.8502514362335205],[What are the benefits of purchasing an Apple ...
freq,1,17,1,1,1,1,1,1,1


In [33]:
texts = df['instruction'].astype(str).tolist()  # Convert to string and then to list

In [34]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Initialize the Tokenizer
token = Tokenizer()

# Fit the Tokenizer on the text data
token.fit_on_texts(texts)

# Get the total number of words
total_words = len(token.word_index) + 1


In [35]:
print("Total words:", total_words)


Total words: 484


In [36]:
n_grams = []
seq_length = 10  # You can adjust this for longer sequences
sequence = token.texts_to_sequences([texts])[0] 
for i in range(seq_length, len(sequence)):
    n_gram_seq = sequence[i-seq_length:i+1]
    n_grams.append(n_gram_seq)

n_grams = np.array(n_grams)

In [38]:
def generate_n_grams(sequence, n):
    n_grams = []
    for i in range(len(sequence) - n + 1):
        n_grams.append(sequence[i:i+n])
    return n_grams

# Example input
sequence = [1, 2, 3, 4, 5]  # Replace with your actual sequence
n = 3  # The value of 'n' for n-grams

n_grams = generate_n_grams(sequence, n)
print(n_grams)

[[1, 2, 3], [2, 3, 4], [3, 4, 5]]


In [39]:
import numpy as np

# Convert the list of n-grams to a NumPy array for easier slicing
n_grams_array = np.array([[1, 2, 3], [2, 3, 4], [3, 4, 5]])

# Split into X (input) and y (target)
X = n_grams_array[:, :-1]  # Take all but the last element of each n-gram
y = n_grams_array[:, -1]   # Take the last element as the target

print("X:", X)
print("y:", y)

X: [[1 2]
 [2 3]
 [3 4]]
y: [3 4 5]


In [40]:
y = to_categorical(y, num_classes=total_words)

In [41]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Defining the model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=64))  # Embedding layer
model.add(LSTM(64))  # Replaced SimpleRNN with LSTM
model.add(Dense(10, activation='softmax'))  # Output layer

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

#model = Sequential()
#model.add(Embedding(total_words, 50, input_length=seq_length))
#model.add(SimpleRNN(150, return_sequences=False))  
#model.add(Dense(100, activation='relu'))
#model.add(Dense(total_words, activation='softmax'))
#Replace simpleRNN with GRU
#from tensorflow.keras.layers import GRU
# model = Sequential()
# model.add(Embedding(total_words, 50, input_length=seq_length))
# model.add(GRU(150, return_sequences=False))  # Replace SimpleRNN with GRU
# model.add(Dense(100, activation='relu'))
# model.add(Dense(total_words, activation='softmax'))

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [42]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def generate_text(seed_text, next_words=50, temperature=1.0):
    for _ in range(next_words):
        token_list = token.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=seq_length, padding='pre')
        
        # Get model predictions
        predictions = model.predict(token_list, verbose=0)
        
        # Apply temperature sampling to the predictions
        predictions = predictions.astype('float64')
        predictions = np.log(predictions + 1e-7) / temperature
        exp_preds = np.exp(predictions)
        predictions = exp_preds / np.sum(exp_preds)
        
        # Choose the next word using a probability distribution
        predicted = np.random.choice(len(predictions[0]), p=predictions[0])
        output_word = token.index_word.get(predicted, '')
        
        # If no valid word is found, stop generating further
        if output_word == '' or (len(seed_text.split()) > 1 and seed_text.split()[-1] == output_word):
            continue
        
        seed_text += " " + output_word
    
    return seed_text

# Generate new text
seed_text = "What are the benefits of"
print(generate_text(seed_text, next_words=10, temperature=1.5))

What are the benefits of that what i what is what the what
